# Teste da API Comtrade

Site: https://comtradedeveloper.un.org
Manual metodológico: https://comtrade.un.org/data/MethodologyGuideforComtradePlus.pdf


API specs: https://comtradedeveloper.un.org/api-details#api=comtrade-v1&operation=get-get



## Descrições dos códigos HS

Obter tabela em https://github.com/datasets/harmonized-system/blob/master/data/harmonized-system.csv

Copiar para directoria `support`


### Ler a tabela

In [1]:
import pandas as pd

hs_codes_df = pd.read_csv('support/harmonized-system.csv')


### Criar um dicionário com os códigos e descrições

In [2]:

hs_codes_map = dict(zip(hs_codes_df.hscode, hs_codes_df.description))

#### Fazer um subconjunto com o nível 2 somente

In [3]:
hs_codes_l2 = hs_codes_df[hs_codes_df.level == 2]
hs_l2_map = dict(zip(hs_codes_l2.hscode, hs_codes_l2.description))



## Obter os dados de comtrade.un.org

### Parâmetros gerais que não mudam

Para os códigos de países e regiões usados ver https://data.apps.fao.org/catalog/dataset/m49-code-list-global-region-country

In [4]:
import json
import requests
import pandas

m49_codes_cn_plp ={
    "024":"Angola",
    "076":"Brazil",
    "132":"Cabo Verde",
    "156":"China",
    "344":"China, Hong Kong",
    "446":"China, Macau",
    "226":"Guiné Equatorial",
    "624":"Guiné-Bissau",
    "508":"Moçambique",
    "620":"Portugal",
    "678":"São Tomé e Príncipe",
    "626":"Timor-Leste"
}
m49_china = "156"
m49_hong_kong = "344"
m49_macau = "446"
m49_plp = set(m49_codes_cn_plp.keys())-{m49_china,m49_hong_kong,m49_macau}
m49_plp_list = ",".join(m49_plp)




### Função auxiliar para aceder à API un.comtrade


In [5]:

def call_uncomtrade(typeCode: str, freqCode: str, 
                    reporterCode: str = '049', 
                    partnerCode: str = '024,076,132,226,624,508,620,678,626',
                    period: str = None,
                    clCode: str = "HS",
                    cmdCode: str = "TOTAL",
                    flowCode: str = "M,X",
                    timeout: int = 10
                    )->pd.DataFrame:
    """ Makes a request to UN Comtrade API (public), returns a pandas DataFrame
    
    Parameters
        typeCode: required, C for commodities, S for Services
        freqCode: required, A for annual and M for monthly
        reporterCode: optional, default "049" (China)
        partnerCode: optional, default list of PLP codes, None for all countries
        period:  optional, aaaa or aaaamm default None (all available periods)
        clCode: Trade classifications: HS, SITC, BEC or EBOPS.
                Available values : HS, SS, B4, B5, EB, EB10, EB02, EBSDMX
        cmdCode: optional, default, "TOTAL"
        flowCode: optional, M=import, X=export,more: RX, RM, MIP, XIP, MOP, XOP, MIF, XIF, DX, FM; default "M,X"
        timeOut: int, max wait time in seconds. Default 10
     """

    baseUrl = "https://comtradeapi.un.org/public/v1"

    requestUrl=f"{baseUrl}/preview/{typeCode}/{freqCode}/{clCode}"
    resp = requests.get(requestUrl,
            {
            'reporterCode':reporterCode,
            'period':period,
            'partnerCode':partnerCode,
            'cmdCode':cmdCode,
            'flowCode':flowCode
            },
            timeout=timeout)
    results = json.loads(resp.content)['data']
    df = pd.DataFrame(results)
    # Convert the country codes in the result to string with leading zero
    # (the country codes are returned as int and lack leading zero)
    df.reporterCode = df.reporterCode.astype('str').str.zfill(3)
    df.partnerCode = df.partnerCode.astype('str').str.zfill(3)
    # Convert the country codes to country names
    df.reporterDesc = df.reporterCode.map(m49_codes_cn_plp)
    df.partnerDesc = df.partnerCode.map(m49_codes_cn_plp)
    # Convert the HS codes
    df.cmdDesc = df.cmdCode.map(hs_codes_map)
    # Generate a formated version of the value for readability here
    df['primaryValueFormated'] = df.primaryValue.map('{:,}'.format)
    # return the DataFrame
    return df

## Parâmetros para visualização

Colunas a visualizar, ordem das linhas



In [9]:
# Colunas mais interessantes do resultado
# escolher de 
#        'typeCode', 'freqCode', 'refPeriodId', 'refYear', 'refMonth',
#        'period', 'reporterCode', 'reporterISO', 'reporterDesc',
#        'flowCode', 'flowDesc', 'partnerCode', 'partnerISO', 'partnerDesc',
#        'partner2Code', 'partner2ISO', 'partner2Desc',
#        'classificationCode', 'classificationSearchCode',
#        'isOriginalClassification', 'cmdCode', 'cmdDesc', 'aggrLevel',
#        'isLeaf', 'customsCode', 'customsDesc', 'mosCode', 'motCode',
#        'motDesc', 'qtyUnitCode', 'qtyUnitAbbr', 'qty', 'isQtyEstimated',
#        'altQtyUnitCode', 'altQtyUnitAbbr', 'altQty', 'isAltQtyEstimated',
#        'netWgt', 'isNetWgtEstimated', 'grossWgt', 'isGrossWgtEstimated',
#        'cifvalue', 'fobvalue', 'primaryValue', 'legacyEstimationFlag',
#        'isReported', 'isAggregate', 'primaryValueFormated']

cols = ['typeCode','freqCode','reporterDesc','partnerDesc','partner2Code','refYear','refMonth','cmdCode','cmdDesc','flowCode','isReported','primaryValueFormated','primaryValue']
sort_order = ['reporterDesc','partnerDesc','refYear','refMonth']

## Notas várias

### Mais do que uma linha por par de países

Alguns anos produzem mais do que uma linha por par _reporter/partner_  com diferentes valores:
2015, 2016, 2017
* Nesses anos existe mais do que uma linha em todos os países, tanto nas exportações como nas importações
* Os países com maior volume têm mais linhas (Brasil, Angola).
  

[_] Verificar se o total do ano quando há duplicação corresponde 
    ao valor reportado em outras fontes, como os relatórios do _Forum_

## Testes

In [10]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

period = "2016" ## if freqCode M  use aaaamm
flow = "X"
partnerCode = "132"
df = call_uncomtrade("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     reporterCode=m49_china,
                     partnerCode=partnerCode,
                     cmdCode='TOTAL',
                     period=period,
                     timeout=20
                     )
df[cols].sort_values(sort_order)

,typeCode,freqCode,reporterDesc,partnerDesc,partner2Code,refYear,refMonth,cmdCode,cmdDesc,flowCode,isReported,primaryValueFormated,primaryValue
0,C,A,China,Cabo Verde,0,2016,52,TOTAL,Total of all HS2022 commodities,X,False,"48,671,438.0",48671438.0
1,C,A,China,Cabo Verde,132,2016,52,TOTAL,Total of all HS2022 commodities,X,False,"48,602,407.0",48602407.0
2,C,A,China,Cabo Verde,344,2016,52,TOTAL,Total of all HS2022 commodities,X,False,"69,027.0",69027.0
3,C,A,China,Cabo Verde,842,2016,52,TOTAL,Total of all HS2022 commodities,X,False,4.0,4.0


In [8]:
print(f"Total para {m49_codes_cn_plp[partnerCode]} {flow} {period} ",df.primaryValue.sum())

Total para Cabo Verde X 2016  97342876.0


In [11]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

df = call_uncomtrade("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     reporterCode=m49_china,
                     partnerCode="132",
                     cmdCode='AG2',
                     period=period,
                     timeout=20
                     )
df[cols].sort_values('primaryValue')

,typeCode,freqCode,reporterDesc,partnerDesc,partner2Code,refYear,refMonth,cmdCode,cmdDesc,flowCode,isReported,primaryValueFormated,primaryValue
133,C,A,China,Cabo Verde,842,2016,52,99,Commodities not specified according to kind,X,False,4.0,4.0
72,C,A,China,Cabo Verde,132,2016,52,16,"Meat, fish, crustaceans, molluscs or other aqu...",X,False,25.0,25.0
7,C,A,China,Cabo Verde,0,2016,52,16,"Meat, fish, crustaceans, molluscs or other aqu...",X,False,25.0,25.0
35,C,A,China,Cabo Verde,0,2016,52,60,Fabrics; knitted or crocheted,X,False,84.0,84.0
100,C,A,China,Cabo Verde,132,2016,52,60,Fabrics; knitted or crocheted,X,False,84.0,84.0
69,C,A,China,Cabo Verde,132,2016,52,11,"Products of the milling industry; malt, starch...",X,False,116.0,116.0
4,C,A,China,Cabo Verde,0,2016,52,11,"Products of the milling industry; malt, starch...",X,False,116.0,116.0
76,C,A,China,Cabo Verde,132,2016,52,22,"Beverages, spirits and vinegar",X,False,225.0,225.0
11,C,A,China,Cabo Verde,0,2016,52,22,"Beverages, spirits and vinegar",X,False,225.0,225.0
70,C,A,China,Cabo Verde,132,2016,52,12,Oil seeds and oleaginous fruits; miscellaneous...,X,False,239.0,239.0


In [30]:
df.primaryValue.sum()

166562.0

### Gravar em Excel

In [72]:
filename_note=period+flow.replace(",","_")  # change to append to filename
df[cols].to_excel(f"./downloads/dados_comtrade_{filename_note}.xlsx")

## Cobertura dos dados: importações/exportações China-PLP anos disponíveis

In [12]:
import time

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

flow = "M,X"
for country_code in m49_plp:
    df = call_uncomtrade("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     reporterCode=m49_china,
                     partnerCode=country_code,
                     period=None # period None returns all the available period
                     )
    print(m49_codes_cn_plp[country_code],  df.refYear.unique())
    time.sleep(1)  # avoid stressing the UN server.



Moçambique [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
Brazil [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
Guiné-Bissau [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
Angola [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
Guiné Equatorial [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
São Tomé e Príncipe [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
Cabo Verde [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
Portugal [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]
Timor-Leste [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]


## China, importações mais importantes dos PLP

In [69]:
rank_filter = 5  # número de importações mais relevantes
years = "2020,2021"
pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdDesc',
            'flowCode','primaryValueFormated']
df = call_uncomtrade("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode="M",
                     cmdCode="AG2",
                     reporterCode=m49_china,
                     partnerCode=m49_plp_list,
                     period=years 
                     )

pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear'])["primaryValue"].rank(method="dense", ascending=False)
pco_top5 = pco[pco['rank'] <= rank_filter]
pco_top5[pco_cols].set_index(['reporterDesc','partnerDesc','refYear'])

rank  \
reporterDesc partnerDesc         refYear         
China        Angola              2020      1.0   
                                 2020      2.0   
                                 2020      3.0   
                                 2020      4.0   
                                 2020      5.0   
                                 2021      1.0   
                                 2021      2.0   
                                 2021      3.0   
                                 2021      4.0   
                                 2021      5.0   
             Brazil              2020      1.0   
                                 2020      2.0   
                                 2020      3.0   
                                 2020      4.0   
                                 2020      5.0   
                                 2021      1.0   
                                 2021      2.0   
                                 2021      3.0   
                                 2021      4.0   
                                 2021      5.0   
             Cabo Verde          2020      1.0   
                                 2020      2.0   
                                 2020      3.0   
                                 2020      4.0   
                                 2020      5.0   
                                 2021      1.0   
                                 2021      2.0   
                                 2021      3.0   
                                 2021      4.0   
                                 2021      5.0   
             Guiné Equatorial    2020      1.0   
                                 2020      2.0   
                                 2020      3.0   
                                 2020      4.0   
                                 2021      1.0   
                                 2021      2.0   
                                 2021      3.0   
                                 2021      4.0   
                                 2021      5.0   
             Guiné-Bissau        2020      1.0   
                                 2020      2.0   
                                 2020      3.0   
                                 2021      1.0   
             Moçambique          2020      1.0   
                                 2020      2.0   
                                 2020      3.0   
                                 2020      4.0   
                                 2020      5.0   
                                 2021      1.0   
                                 2021      2.0   
                                 2021      3.0   
                                 2021      4.0   
                                 2021      5.0   
             Portugal            2020      1.0   
                                 2020      2.0   
                                 2020      3.0   
                                 2020      4.0   
                                 2020      5.0   
                                 2021      1.0   
                                 2021      2.0   
                                 2021      3.0   
                                 2021      4.0   
                                 2021      5.0   
             São Tomé e Príncipe 2020      1.0   
                                 2020      2.0   
                                 2020      3.0   
                                 2020      4.0   
                                 2020      5.0   
                                 2021      1.0   
                                 2021      2.0   
                                 2021      3.0   
                                 2021      4.0   
                                 2021      5.0   
             Timor-Leste         2020      1.0   
                                 2020      2.0   
                                 2020      3.0   
                                 2020      4.0   
                                 2020      5.0   
                                 2021     

### Guardar formato excel


In [73]:
filename_note=years  # change to append to filename
pco_top5.to_excel(f"./downloads/china_plp_import_top5_{filename_note}.xlsx")

## China exportações mais importantes para os PLP

In [80]:
rank_filter = 5  # número de importações mais relevantes
years = "2021"
pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdDesc',
            'flowCode','primaryValueFormated']
df = call_uncomtrade("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode="X",
                     cmdCode="AG2",
                     reporterCode=m49_china,
                     partnerCode=m49_plp_list,
                     period=years 
                     )

pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear'])["primaryValue"].rank(method="dense", ascending=False)
pco_top5 = pco[pco['rank'] <= rank_filter]
pco_top5[pco_cols].set_index(['reporterDesc','partnerDesc','refYear'])

rank  \
reporterDesc partnerDesc      refYear         
China        Angola           2021      1.0   
                              2021      2.0   
                              2021      3.0   
                              2021      4.0   
                              2021      5.0   
             Brazil           2021      1.0   
                              2021      2.0   
                              2021      3.0   
                              2021      4.0   
                              2021      5.0   
             Cabo Verde       2021      1.0   
                              2021      2.0   
                              2021      3.0   
                              2021      4.0   
                              2021      5.0   
             Guiné Equatorial 2021      1.0   
                              2021      2.0   
                              2021      3.0   
                              2021      4.0   
                              2021      5.0   
             Guiné-Bissau     2021      1.0   
                              2021      2.0   
                              2021      3.0   
                              2021      4.0   
                              2021      5.0   
             Moçambique       2021      1.0   
                              2021      2.0   
                              2021      3.0   
                              2021      4.0   
                              2021      5.0   
             Portugal         2021      1.0   
                              2021      2.0   
                              2021      3.0   
                              2021      4.0   
                              2021      5.0   

                                                                                 cmdDesc  \
reporterDesc partnerDesc      refYear                                                      
China        Angola           2021     Electrical machinery and equipment and parts t...   
                              2021     Machinery and mechanical appliances, boilers, ...   
                              2021     Vehicles; other than railway or tramway rollin...   
                              2021                         Plastics and articles thereof   
                              2021     Textiles, made up articles; sets; worn clothin...   
             Brazil           2021     Electrical machinery and equipment and parts t...   
                              2021     Machinery and mechanical appliances, boilers, ...   
                              2021                                     Organic chemicals   
                              2021                                        Iron and steel   
                              2021     Optical, photographic, cinematographic, measur...   
             Cabo Verde       2021     Meat, fish, crustaceans, molluscs or other aqu...   
                              2021     Machinery and mechanical appliances, boilers, ...   
                              2021     Electrical machinery and equipment and parts t...   
                              2021     Optical, photographic, cinematographic, measur...   
                              2021                        Aluminium and articles thereof   
             Guiné Equatorial 2021     Machinery and mechanical appliances, boilers, ...   
                              2021                         Plastics and articles thereof   
                              2021                               Pharmaceutical products   
                              2021                                Iron or steel articles   
                              2021     Electrical machinery and equipment and parts t...   
             Guiné-Bissau     2021                                               Cereals   
                              2021                               Pharmaceutical products   
                              2021                              Chemical products n.e.c.   

### Guardar formato excel


In [76]:
filename_note=years  # change to append to filename
pco_top5.to_excel(f"./downloads/china_plp_export_top5_{filename_note}.xlsx")

### China, detalhe das exportações mais importants dos PLP para a China

Em dois passos: 
* obtém as categorias de nível 2 mais importantes de cada país
* pesquisa todos as subcategorias de cada

In [81]:
rank_filter = 5  # número de importações mais relevantes
years = "2020,2021"
pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdDesc',
            'flowCode','primaryValueFormated']
df = call_uncomtrade("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode="M",
                     cmdCode="AG2",
                     reporterCode=m49_china,
                     partnerCode=m49_plp_list,
                     period=years 
                     )

pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear'])["primaryValue"].rank(method="dense", ascending=False)
pco_top5 = pco[pco['rank'] <= rank_filter]
# get the countries
countries = pco_top5.partnerDesc.unique()
country_cmd_top5_codes = dict()
for country in countries:
    l2_codes = pco_top5[pco_top5.partnerDesc == country]['cmdCode'].unique()
    print(country,l2_codes)
    hs_details = []
    for l2_code in l2_codes:
        l2_sub_codes = list(hs_codes_df[hs_codes_df.hscode.str.startswith(l2_code)]['hscode'])
        hs_details = hs_details + l2_sub_codes
    # print(hs_details)
    country_cmd_top5_codes[country] = hs_details.copy()


Angola ['27' '25' '71' '74' '26' '76']
Brazil ['26' '12' '27' '02' '47' '17']
Cabo Verde ['74' '84' '85' '99' '63' '26' '62']
Guiné Equatorial ['27' '44' '99' '63' '90' '84']
Guiné-Bissau ['49' '99' '85']
Moçambique ['26' '44' '12' '74' '71' '27']
Portugal ['85' '87' '61' '84' '25' '74']
São Tomé e Príncipe ['84' '90' '73' '85' '39' '99']
Timor-Leste ['12' '09' '38' '85' '44' '27' '14']


Exemplo dos códigos relevantes para o detalhe de Angola

In [83]:
country_cmd_top5_codes['Angola']

['27',
 '2701',
 '270111',
 '270112',
 '270119',
 '270120',
 '2702',
 '270210',
 '270220',
 '2703',
 '270300',
 '2704',
 '270400',
 '2705',
 '270500',
 '2706',
 '270600',
 '2707',
 '270710',
 '270720',
 '270730',
 '270740',
 '270750',
 '270791',
 '270799',
 '2708',
 '270810',
 '270820',
 '2709',
 '270900',
 '2710',
 '271012',
 '271019',
 '271020',
 '271091',
 '271099',
 '2711',
 '271111',
 '271112',
 '271113',
 '271114',
 '271119',
 '271121',
 '271129',
 '2712',
 '271210',
 '271220',
 '271290',
 '2713',
 '271311',
 '271312',
 '271320',
 '271390',
 '2714',
 '271410',
 '271490',
 '2715',
 '271500',
 '2716',
 '271600',
 '25',
 '2501',
 '250100',
 '2502',
 '250200',
 '2503',
 '250300',
 '2504',
 '250410',
 '250490',
 '2505',
 '250510',
 '250590',
 '2506',
 '250610',
 '250620',
 '2507',
 '250700',
 '2508',
 '250810',
 '250830',
 '250840',
 '250850',
 '250860',
 '250870',
 '2509',
 '250900',
 '2510',
 '251010',
 '251020',
 '2511',
 '251110',
 '251120',
 '2512',
 '251200',
 '2513',
 '251310',